In [66]:
import numpy as np
import csv
import plotly
import plotly.graph_objs as go
from PIL import Image, ImageDraw, ImageFont
from scipy.ndimage import gaussian_filter, gaussian_filter1d
from math import asin, acos, degrees, sin, cos, pi

plotly.offline.init_notebook_mode(connected=True)
py = plotly.offline

# read data

def convert(row):
    id = row[0]
    rssi = row[1]
    x = row[2]
    y = row[3]
    return (id, float(rssi), (float(x) + 390) / 3, (float(y) + 960) / 3)

beacon_ids = ['cd', 'd7', '17', '51', '43', 'b8', '2a', 'f8', '3d', '62',
            '3']
    
def read_data():
    with open('rssi_sat.csv', 'rb') as f:
        reader = csv.reader(f)
        data = map(convert, reader)

    data_for_beacon = {}
    for id in beacon_ids:
        data_for_beacon[id] = []
        
    for row in data:
        beacon = row[0]
        data_for_beacon[beacon].append(row)

    return data_for_beacon

data_for_beacon = read_data()

pixel_origin = np.array([125, 410])
global_origin = np.array([55.94455753546212,-3.1866420060396194])

coor_change = np.array([[-2.62187e6, 422601.], [-799717., -1.41637e6]])

# convert global coordinates to pixels

beacons_global = np.array([
    [55.9444578385393,-3.1866151839494705],
    [55.94444244275808,-3.18672649562358860],
    [55.94452336441765,-3.1866540759801865],
    [55.94452261340533,-3.1867526471614838],
    [55.94448393625199,-3.1868280842900276],
    [55.94449050761571,-3.1866483762860294],
    [55.94443774892113,-3.1867992505431175],
    [55.944432116316044,-3.186904862523079],
    [55.94444938963575,-3.1869836524128914],
    [55.94449107087541,-3.186941407620907]
])

beacons_pixel = np.array(np.transpose(coor_change.dot(np.transpose(beacons_global - global_origin))) + pixel_origin, dtype=np.int16)

def rssi_from_data(data):
    return np.array([row[1] for row in data], dtype=np.float)

def pos_from_data(data):
    return np.array([[row[2], row[3]] for row in data], dtype=np.float)

def dist_from_point(point, positions):
    diff = positions - point
    return np.linalg.norm(diff, axis=1)

class Beacon(object):
    def __init__(self, b_id, pos):
        self.name = b_id
        self.pos = pos
        self.points = pos_from_data(data_for_beacon[b_id])
        self.rssi = rssi_from_data(data_for_beacon[b_id])
        self.dist = dist_from_point(pos, self.points)
        self.f_dist_rssi = np.poly1d(np.polyfit(self.rssi, self.dist, 2))
        self.f_rssi_dist = np.poly1d(np.polyfit(self.dist, self.rssi, 2))
        
    def norm(self, point):
        return np.linalg.norm(point - self.pos)
        
b_17 = Beacon('17', beacons_pixel[2])
b_b8 = Beacon('b8', beacons_pixel[5])

def gamma(a, b, c):
    cosval = (a ** 2 + b ** 2 - c ** 2)/(2.0 * a * b)
    if cosval > 1.:
        cosval = 1.
    elif cosval < -1.:
        cosval = -1.
    return acos(cosval)

v = b_b8.pos - b_17.pos
r = np.linalg.norm(v)
alpha = asin(v[1] / v[0])

def get_position_from_dist(r_17, r_b8):    
    beta = gamma(r_17, r, r_b8)
    return b_17.pos + [r_17 * cos(alpha + beta), r_17 * sin(alpha + beta)]

def read_data_17_b8():
    with open('rssi_sat.csv', 'rb') as f:
        reader = csv.reader(f)
        rssi_17_b8 = []
        positions_17_b8 = []
        last_17 = -70
        last_b8 = -70
        for [b_id, rssi, x, y] in reader:
            rssi, x, y = int(rssi), (int(x) + 390.) / 3, (int(y) + 960.) / 3
            if b_id == '17' and rssi > -80:
                last_17 = rssi
                rssi_17_b8.append(np.array([last_17, last_b8], dtype=np.float))
                positions_17_b8.append(np.array([x, y], dtype=np.float))
            elif b_id == 'b8' and rssi > -80:
                last_b8 = rssi
                rssi_17_b8.append(np.array([last_17, last_b8], dtype=np.float))
                positions_17_b8.append(np.array([x, y], dtype=np.float))

    return (np.array(rssi_17_b8, dtype=np.float), np.array(positions_17_b8, dtype=np.float))

rssi_17_b8, positions_17_b8 = read_data_17_b8()

def get_rel_dist_from_17_b8(rssi):
    return np.array([b_17.f_dist_rssi(rssi[0]), b_b8.f_dist_rssi(rssi[1])])

def get_pos_from_rssi(rssi):
    return get_position_from_dist(*get_rel_dist_from_17_b8(rssi))

def get_rssi_from_pos(pos):
    return np.array([b_17.f_rssi_dist(b_17.norm(pos)), b_b8.f_rssi_dist(b_b8.norm(pos))])

def estimate_position(rssi):
#     smooth_rssi = gaussian_filter1d(rssi, sigma=16, axis=0)
    return np.apply_along_axis(get_pos_from_rssi, 1, rssi)

def estimate_rssi(pos):
    return np.apply_along_axis(get_rssi_from_pos, 1, pos)

In [74]:
# np.cov(rssi_17_b8.T)

def plot_rssi_actual_and_predicted(actual, predicted, start, n):
    stop = start + n
    domain = np.linspace(0, 1, n)
    
    def plot(i, name):
        py.iplot({
            "data": [
                go.Scatter(x=domain, y=actual[start:stop, i], name='actual'),
                go.Scatter(x=domain, y=predicted[start:stop, i], name='predicted')
            ],
            "layout": go.Layout(
                title=name,
                yaxis=dict(
                    range=[-100, -50]
                )
            )
        })
        
    plot(0, '17')
    plot(1, 'b8')

    
plot_rssi_actual_and_predicted(rssi_17_b8, estimate_rssi(positions_17_b8), 0, 3100)
len(rssi_17_b8)

3057

now compute

[rssi, rssi] is the state
[rssi, rssi] is the observation

ground truth is what follows from our f_rssi_dist function

we have the measurements and and the positions
compute the ideal rssi from positions, and plot with actual rssi